<a href="https://colab.research.google.com/github/Deepprakash222/DeepGpt3Learning/blob/master/DDL_Hackashon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kings' Bot

In [ ]:
#@title Installing libraries
!pip install -q openai

In [ ]:
#@title Libraries

import sys
import os
import re
from typing import List, Text, Dict, Tuple
import openai
from getpass import getpass
from pprint import pprint

import nltk
nltk.download('punkt')
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#@title Write down your API
api = getpass('Write OpenAPI Hash')
os.environ['OPENAI_API_KEY'] = api


In [ ]:
history_chat = ["King is an intelligent assistant author of horror, supernatural fiction, suspense, crime, science-fiction, and fantasy novels inspired by Stephen King. It has great ideas for articles, blogs, and movie creation. It is helping to provide article titles, sub-headers for the article's title, and also great, flourishing, expensive vocabulary within paragraphs. If the user said no to something, King will offer to help the user.\n\nKing: Hello there, you have with you, King! an Artificial Intelligent Assistant for helping you create beautiful, inspired, horror, supernatural fiction, suspense, crime, science-fiction, fantasy articles, and novels. Please tell me if you have an idea for a title for what you want to write about.\n"]
ARTICLE = []

In [ ]:
# history_chat = ["King is an intelligent assistant author of horror, supernatural fiction, suspense, crime, science-fiction, and fantasy novels inspired by Stephen King. It has great ideas for articles, blogs, and movie creation. It is helping to provide article titles, sub-headers for the article's title, and also great, flourishing, expensive vocabulary within paragraphs. If the user said no to something, King will offer to help the user.\n\nKing: Hello there, you have with you, King! an Artificial Intelligent Assistant for helping you create beautiful, inspired, horror, supernatural fiction, suspense, crime, science-fiction, fantasy articles, and novels. Please tell me if you have an idea for a title for what you want to write about.\n"]
# ARTICLE = []

openai.api_key = os.getenv("OPENAI_API_KEY")

def chatbot(user_insertion: Text):
  global history_chat
  history_chat.append(f"You: {user_insertion}\nKing:")
  response = openai.Completion.create(
    model="text-davinci-002",
    prompt="".join(history_chat),
    temperature=0.85,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
  response = re.sub(r"(^\n\n)(\S.+)", r"\2", response.choices[0].text)
  history_chat.append(f"{response}\n")
  # print(f"Answer:{response}\n\nHistory:\n")
  # print("".join(history_chat))

  return response


def paragraph_generator(list_of_sub_headers: List):
  list_of_paragraphs = []
  list_of_sub_headers = list(filter(None, list_of_sub_headers[-1].strip(' ').split('\n')))

  prompt = ["Expand these sub-headers into a detailed professional, witty and clever explanation paragraph."]

  for sub_header in tqdm(list_of_sub_headers, desc="Generating paragraphs"):
    # print(sub_header)
    prompt.append(f'\n\n"{sub_header}"\n')
    
    response = openai.Completion.create(
      model="text-davinci-002",
      prompt="".join(prompt),
      temperature=0.7,
      max_tokens=200,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    list_of_paragraphs.append(
        sentence_tokenization(
            re.sub(r"("+sub_header+")", "", response.choices[0].text.replace(sub_header, ""),
                   re.IGNORECASE)
        )
    )

  return "\n\n".join("".join([header, paragraph])
                 for header, paragraph in  list(zip(list_of_sub_headers,
                                                    list_of_paragraphs)))


def sentence_tokenization(text: Text) -> List[Text]:
  text = re.sub(r"(^\n)(\n)(\w)", r"\2\3", text)
  tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
  str_tokenized_text = " ".join([sentence for sentence in tokenizer.tokenize(text) if sentence[-1] in [".", "!", "?"]])
  str_tokenized_text = str_tokenized_text.replace(r'\s ', ' ')
  str_tokenized_text= str_tokenized_text.replace(r'(^\s)|(\s$)', '')
  return str_tokenized_text



def store_content(user_insertion: Text):
  if re.search(r'(title approved)', user_insertion, re.IGNORECASE):
    ARTICLE.append(history_chat[-1])
    return chatbot(user_insertion)
  elif re.search(r'(sub-headers approved)', user_insertion, re.IGNORECASE):
    ARTICLE.append(history_chat[-1])
    return chatbot(user_insertion)
  elif re.search(r'(expand sub-headers)', user_insertion, re.IGNORECASE):
    #TODO we need to add a progress bar or a progress event for this function
    ARTICLE.append(paragraph_generator(ARTICLE))
    return "Sorry if it took much time. I hope you would like the content of each sub-header."
  elif re.search(r'(content approved)', user_insertion, re.IGNORECASE):
    return "I'm happy that I was able to help you. Please give us feedback about your satisfaction – this would mean a lot to me as an author."
  else:
    return chatbot(user_insertion)

## Run the prototype

In [ ]:
store_content("expand sub-headers")

##For intermediate tests

In [ ]:
paragraph_generator(ARTICLE)

In [ ]:
list(filter(None, ARTICLE[-1].strip(' ').split('\n')))

In [ ]:
pprint(ARTICLE)